# Import Packages

In [1]:
!pip install mat73

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm

import tensorflow as tf
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, LayerNormalization
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers import LeakyReLU 
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam, RMSprop, Adadelta, SGD
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay

import sys

from scipy.stats import gaussian_kde as kde
import plotly.graph_objects as go
import mat73

# Load Data

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

topic = 'Ming/Public_Data'
work_dir = '/content/gdrive/My Drive/Colab Notebooks/' + topic
import os
os.chdir(work_dir)
!pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/Ming/Public_Data


In [5]:
################### Data mat ####################
'''
Experiment 1: Gaussian_steady_state_512.mat
Experiment 2: Exponential_transient_256.mat
Experiment 3: Gaussian_steady_state_3D.mat
Experiment 4: Channel_Binary_steady_state_128.mat
Experiment 5: Channel_Continuous_steady_state_128.mat
'''
cov_type = 'Gaussian'
pump_type = 'steady_state'
ext = '3D.mat'
data = mat73.loadmat('./Data/'+'_'.join([cov_type,pump_type,ext]))


In [7]:
npump=25
pump_well_id = np.arange(npump,dtype=int)

# geometric mean of conductivity 
mu = int(data['mu'])

# variance of log-conductivity
sigma2= int(data['sigma2'])

# domain length
Lx = data['Lx']

# correlaton length in ratio (0,1)
lxr = data['lxr']

# resolutions
nx = data['nx'].astype(int)

# PCA componenets
Z = data['Z'].T

# latent variables
alpha = data['rv'].T

# number of realizations
NR = alpha.shape[0]

# monitored hydraulic heads
y_save_monitored = data['y_save_monitored'].reshape((NR,25,16,npump),order='F')

Y = np.zeros((NR,24,16,25))

for ii in range(Y.shape[-1]):
  Y[:,:,:,ii] = np.delete(y_save_monitored[:,:,:,ii],pump_well_id[ii],axis=1)

Y = Y.reshape([Y.shape[0],-1])

y_save = data['y_save'].reshape((10, nx[0], nx[1], nx[2],npump), order='F')

[320. 320.  16.]


In [19]:
lxr1, lxr2, lxr3 = lxr
Lx1, Lx2, Lx3 = Lx 
nx1, nx2, nx3 = nx

# correlation length
lx1 = lxr1*Lx1
lx2 = lxr2*Lx2
lx3 = lxr3*Lx3 
lx = [lx1, lx2, lx3]

# grid spacing
dx1 = Lx1 / nx1 
dx2 = Lx2 / nx2
dx3 = Lx3 / nx3
dx = [dx1,dx2,dx3]

# Exponential (1) or Gaussian (2) Covariance-Model 
Ctype = 1

# tot elements
ntot=np.prod(nx)

In [24]:
# ########################################################
Y_noised = np.zeros_like(Y)
# Y_noised = Y
for i in range(Y.shape[0]):
  Y_noised[i]  = Y[i] + np.random.normal(0,np.abs(Y[i]*0.05),[1,Y[i].shape[0]])
train_test_cut = int(0.9*alpha.shape[0])
X_train, X_test = alpha[:train_test_cut], alpha[train_test_cut:]
Y_train, Y_test = Y_noised[:train_test_cut], Y_noised[train_test_cut:]
print(X_train.shape)
print(Y_train.shape)

(4500, 50)
(4500, 9600)


# PCA Decode

In [9]:
ss_train = np.reshape(alpha @ Z, (NR, nx[0], nx[1], nx[2]), order='F')

In [32]:
x_ = np.linspace((-Lx[0]/2+dx[0]/2),(Lx[0]/2-dx[0]/2),nx[0])
y_ = np.linspace((-Lx[1]/2+dx[1]/2),(Lx[1]/2-dx[1]/2),nx[1])
z_ = np.linspace((-Lx[2]+dx[2]/2),-dx[0]/2,nx[2])
[Xm,Ym,Zm]=np.meshgrid(x_, y_, z_)


## Gaussian Conductivity field

In [14]:

fig= go.Figure(data=go.Isosurface(
    x=Xm.flatten(),
    y=Ym.flatten(),
    z=Zm.flatten(),
    value=ss_train[0,...].flatten(),
    caps=dict(x_show=False, y_show=False, z_show=False),
    colorscale='Jet',
    slices_x=dict(show=True, locations=[0-Lx[0]*0.25, 0+Lx[0]*0.25]),
    slices_y=dict(show=True, locations=[0-Lx[1]*0.25, 0+Lx[1]*0.25]),
    slices_z=dict(show=True, locations=[0-Lx[2]*0.25, 0-Lx[2]*0.75])
))

fig.update_layout(
    title='True Realization',
    autosize=False,
    width=400, height=300,
    margin=dict(l=0, r=0, t=40, b=40),
    scene=dict(aspectmode='manual',
    aspectratio=dict(x=1.0,y=1.0,z=0.3)),

)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=0.1,
  )

fig.show()

Output hidden; open in https://colab.research.google.com to view.

## Hydraulic head contour

In [15]:

fig= go.Figure(data=go.Isosurface(
    x=Xm.flatten(),
    y=Ym.flatten(),
    z=Zm.flatten(),
    value=y_save[0,...,0].flatten(),
    surface_count=10, # number of isosurfaces, 2 by default: only min and max
    colorbar_nticks=5, # colorbar ticks correspond to isosurface values
    caps=dict(x_show=False, y_show=False, z_show=False),
    colorscale='Viridis',
))

fig.update_layout(
    title='True Realization',
    autosize=False,
    width=400, height=300,
    margin=dict(l=0, r=0, t=40, b=40),
    scene=dict(aspectmode='manual',
    aspectratio=dict(x=1.0,y=1.0,z=0.3)),
)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=0.1,
  )

fig.show()

Output hidden; open in https://colab.research.google.com to view.

# Define Predictor FCNN Model

Input: Heads (normalized)

Output: Latent variables (alpha)


In [21]:
hidden_dim = 3000
hidden_normal = LayerNormalization(axis=1)
input = Input(shape = (Y_train.shape[-1],))

g = BatchNormalization()(input)

g = Dense(hidden_dim)(g)
g = BatchNormalization()(g)
g = LeakyReLU(alpha=0.2)(g)

g = Dense(hidden_dim)(g)
g = BatchNormalization()(g)
g = LeakyReLU(alpha=0.2)(g)

g = Dense(hidden_dim)(g)
g = BatchNormalization()(g)
g = LeakyReLU(alpha=0.2)(g)

g = Dense(hidden_dim)(g)
g = BatchNormalization()(g)
g = LeakyReLU(alpha=0.2)(g)

g_out = Dense(50)(g)
g_out = LayerNormalization(axis=1)(g_out)

G = Model(input, g_out, name = 'experiment_name')
G.summary()

alpha_loss_weight = np.ones((50,))

def weighted_mse(weights=alpha_loss_weight):
  def loss(y_true, y_pred):
    squared_difference_1 = tf.square(y_true - y_pred)*weights
    total_loss = tf.reduce_mean(squared_difference_1, axis=-1)
    return total_loss
  return loss

step = tf.Variable(0, trainable=False)
boundaries = [10000, 20000]
values = [1e-3, 1e-4, 1e-5]
learning_rate_fn = PiecewiseConstantDecay(
    boundaries, values)

# Later, whenever we perform an optimization step, we pass in the step.
learning_rate = learning_rate_fn(step)

G.compile(loss=weighted_mse(), optimizer=Adam(learning_rate, beta_1=0.5))

Model: "experiment_name"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 9600)]            0         
                                                                 
 batch_normalization_5 (Batc  (None, 9600)             38400     
 hNormalization)                                                 
                                                                 
 dense_5 (Dense)             (None, 3000)              28803000  
                                                                 
 batch_normalization_6 (Batc  (None, 3000)             12000     
 hNormalization)                                                 
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 3000)              0         
                                                                 
 dense_6 (Dense)             (None, 3000)          

## Train Model

In [25]:
import time
batch_size = 1000

epochs = 50000
print_interval = 1
start_time = time.time()
for epoch in range(epochs):
  idx = np.random.randint(0, Y_train.shape[0], batch_size)
  to_train = Y_train[idx]
  expect_output = X_train[idx]

  loss = G.train_on_batch(to_train, expect_output)
  if epoch % print_interval == 0:
    colapsed_time = time.time()
    print("[epoch %d], [the loss is %f], [the time is %f]" % (epoch, loss, colapsed_time-start_time))
  start_time = time.time()
  if loss < 1e-5:
    break


KeyboardInterrupt: ignored

## Load saved model

In [ ]:
# '_5000_normalized':0.377
# '_5000_512_normalized':0.99
# '_5000_512':0.87
# '_5000':0.36
# steady_state: 0.37
model_name = './head_to_latent_model/'+field_type+'_'+pumping_type+'_3D'
# G = load_model(model_name)
G = load_model(model_name, custom_objects={ 'loss': weighted_mse(alpha_loss_weight) })
# G.summary()

# Inverse on test data set

In [33]:
predictions = G.predict(Y_test,verbose=0)

pred_imgs = np.matmul(predictions,Z).reshape((500, nx[0], nx[1], nx[2]), order='F')
true_imgs = np.matmul(X_test,Z).reshape((500, nx[0], nx[1], nx[2]), order='F')


In [34]:
acc_list = np.zeros((1,Y_test.shape[0]))
thres = 0.15

for check_id in range(Y_test.shape[0]):

  logK_pred = pred_imgs[check_id,...]
  logK_true = true_imgs[check_id,...]
  K_len = np.max(logK_true) - np.min(logK_true)

  res = abs(logK_true-logK_pred)/K_len
  acc = np.sum(res<thres)/(ntot)

  acc_list[0,check_id] = acc

print("Min Acc: ", np.min(acc_list))
print("Max Acc: ", np.max(acc_list))
print("Arg Max Acc: ", np.argmax(acc_list))
print("Over 90%: ", acc_list[acc_list>=0.9].shape[0])

print("mean Acc: ", np.mean(acc_list))
print("std Acc: ", np.std(acc_list))



Min Acc:  0.27947998046875
Max Acc:  0.6067352294921875
Arg Max Acc:  28
Over 90%:  0
mean Acc:  0.41898199462890623
std Acc:  0.06171858289699025
[205 154 407  51  28]


## Best Estimate

In [38]:
fid = np.argmax(acc_list)

logK_true = true_imgs[fid,:]
logK_pred = pred_imgs[fid,:]

fig= go.Figure(data=go.Isosurface(
    x=Xm.flatten(),
    y=Ym.flatten(),
    z=Zm.flatten(),
    value=logK_true.flatten(),
    surface_fill=0.4,
    caps=dict(x_show=False, y_show=False, z_show=False),

    slices_x=dict(show=True, locations=[0-Lx[0]*0.25, 0+Lx[0]*0.25]),
    slices_y=dict(show=True, locations=[0-Lx[1]*0.25, 0+Lx[1]*0.25]),
    slices_z=dict(show=True, locations=[0-Lx[2]*0.25, 0-Lx[2]*0.75]),
    colorscale='Jet'
))
fig.update_layout(
    title='True Realization',
    autosize=False,
    width=400, height=300,
    margin=dict(l=0, r=0, t=40, b=40),
    scene=dict(aspectmode='manual',
    aspectratio=dict(x=1.0,y=1.0,z=0.3)),

)
fig.show()


Output hidden; open in https://colab.research.google.com to view.

In [39]:
fig= go.Figure(data=go.Isosurface(
    x=Xm.flatten(),
    y=Ym.flatten(),
    z=Zm.flatten(),
    value=logK_pred.flatten(),
    surface_fill=0.4,
    caps=dict(x_show=False, y_show=False, z_show=False),

    slices_x=dict(show=True, locations=[0-Lx[0]*0.25, 0+Lx[0]*0.25]),
    slices_y=dict(show=True, locations=[0-Lx[1]*0.25, 0+Lx[1]*0.25]),
    slices_z=dict(show=True, locations=[0-Lx[2]*0.25, 0-Lx[2]*0.75]),
    colorscale='Jet'
))

fig.update_layout(
    title='Estimation',
    autosize=False,
    width=400, height=300,
    margin=dict(l=0, r=0, t=40, b=40),
    scene=dict(aspectmode='manual',
    aspectratio=dict(x=1.0,y=1.0,z=0.3)),

)
fig.show()

Output hidden; open in https://colab.research.google.com to view.